In [14]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

# Run Queries

In [15]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [16]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [17]:
#clear_buyers[(clear_buyers.is_kam == 0) & (clear_buyers.business_type == "Restaurante")].customer_id.unique()

In [18]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
frida_products = frida_products[['upc','id_tarjeta','categoria','subcategoria']].copy()

df_margen = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144493,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margen = pd.merge(df_margen, frida_products,  how='left', left_on=['sku'], right_on = ['upc'])
df_margen[["gmv_value","cogs_value"]] = df_margen[["gmv_value","cogs_value"]].astype(float)

200
200


In [19]:
# calculate the profit margin by sku
df_margen2 = df_margen.groupby(by=['region','id_tarjeta']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum() if x['gmv_value'].sum() != 0 else 0).reset_index(name='GM')

# calculate the profit margin by subcategory
df_margen3 = df_margen.groupby(by=['region','subcategoria']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum()).reset_index(name='GM')

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [20]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    bcs.customer_segment_id AS customer_segment_id_all,
    bcs2.customer_segment_id AS customer_segment_id_control,
    bcs3.customer_segment_id AS customer_segment_id_test,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON (replace(bcs.customer_segment_name,'ALL_MM1','CONTROL_MM1') = bcs2.customer_segment_name)-- & (bcs2.customer_segment_name ILIKE '%2023-03-16%')
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs3    ON (replace(bcs.customer_segment_name,'ALL_MM1','TEST_MM1') = bcs3.customer_segment_name)-- & (bcs3.customer_segment_name ILIKE '%2023-03-16%')

WHERE offer_name ILIKE '%hoo%'
    AND end_date >= date(getdate()) - 7
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    AND bs.site_identifier_value = '{ciudad}'
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_all,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(ciudad=city_code)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [21]:
df_offers = df_offers.loc[~df_offers.offer_id.isin([-14495056,-14495055])].reset_index(drop=True)

In [22]:
df_offers

,region_code,offer_id,customer_segment_id_all,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name,cat,subcat
0,CMX,-14497887,134056,82052,82252,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
1,CMX,-14497887,134056,54302,82252,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
2,CMX,-14497887,134056,82252,53252,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
3,CMX,-14497887,134056,62702,82052,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
4,CMX,-14497887,134056,58002,31054,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,CMX,-14497887,134056,53102,62704,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
407,CMX,-14497887,134056,51702,31054,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
408,CMX,-14497887,134056,57852,54252,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas
409,CMX,-14497887,134056,103902,134056,Hook_CMX_Aceite_Vegetal_Cristal__x_1_lt_1_%Mar...,2023-03-14,2023-04-01,1.0,45720,Aceite Vegetal Cristal x 1 lt,Abarrotes,Aceites & Grasas


In [23]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    --ffg.close_date
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('week', date(getdate()) - interval '4 week')
    --AND bo.submit_date >= DATE_TRUNC('week', DATE_TRUNC('day', TO_TIMESTAMP('2023-02-15', 'YYYY-MM-DD')) - INTERVAL '15 days')
    --AND bo.submit_date < TO_TIMESTAMP('2023-03-10', 'YYYY-MM-DD')
    --AND extract(year from ffg.close_date) = extract(year from CURRENT_DATE) 
    --AND extract(month from ffg.close_date) >= extract(month from CURRENT_DATE) - 2
    --AND extract(month from ffg.close_date) <= extract(month from CURRENT_DATE) + 1
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{ciudad}'
    
--GROUP BY bo.order_id, ffg.close_date
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    TO_CHAR(DATE_TRUNC('week', DATE_TRUNC('day', bo.submit_date) + INTERVAL '3 day'), 'WW') AS week,
    --TO_CHAR(bo.submit_date, 'WW') AS week,
    bo.customer_id,
    bs2.name,
    bs.addl_product_id as padre_sku_id,
    bcat2.name AS category,
    bcat.name AS subcat,
    MIN(bo.submit_date) AS min_date,
    COUNT(DISTINCT bo.order_id) as orders,
    CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    CASE WHEN s.site_identifier_value = 'CMX' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    ELSE (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/3776) END AS discount_usd,
    SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    --AND LOWER(boida.adjustment_reason) ILIKE '%merma%'
    
GROUP BY 1,2,3,4,5,6,7
""".format(ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd","discount_usd","cant"]] = data_ventas_aux[["gmv_usd","discount_usd","cant"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])
#data_ventas_aux = data_ventas_aux.loc[data_ventas_aux.week.isin(['05','06','07'])].reset_index(drop=True)

In [24]:
data_ventas_aux.head()

,city,week,customer_id,name,padre_sku_id,category,subcat,min_date,orders,gmv_usd,discount_usd,cant
0,CMX,09,15962831,Zanahoria,-201392,Frutas & Verduras,Verduras,2023-03-01 01:51:28.363,1,0.99643,0.13944,2.0
1,CMX,09,99830021,Limón con semilla,-201429,Frutas & Verduras,Frutas,2023-03-01 21:49:11.665,1,4.71755,0.37709,3.0
2,CMX,10,-260670,Zanahoria,-201392,Frutas & Verduras,Verduras,2023-03-03 19:20:46.502,3,10.53433,0.42035,22.0
3,CMX,10,23690458,Huevos Blancos Medianos x 360 und,408867,Lácteos & Huevos,Huevos,2023-03-08 02:49:06.299,1,51.89770,3.11399,1.0
4,CMX,11,52560932,Pan para Hot Dog Medias Noches Bimbo 16 und,289018,Abarrotes,Panadería & Tortilleria,2023-03-16 20:38:16.759,1,2.94758,0.29465,1.0


In [25]:
data_ventas_aux.groupby('week').min_date.min()

week
09   2023-02-27 00:00:25.784
10   2023-03-03 00:01:01.148
11   2023-03-10 00:01:05.937
12   2023-03-17 00:00:16.002
13   2023-03-24 00:00:01.047
Name: min_date, dtype: datetime64[ns]

# Cleaning Data and some functions

In [26]:
if city_code == 'BOG':
    skus_exp = [180632, -200532, 184859, 354712, 354711, 440936, -200916, 78522, 204584, 118709, 193562, 19739, 193561, 150836, 241860, 143959, 26182, 136259, 104609]
elif city_code == 'SPO':
    skus_exp = [-202917, 60544, 84008, 197110, 383859, 421531, 210067, 205461, 123459, 139969, 293209, 313963, 59581, 313965]
elif city_code == 'CMX':
    skus_exp = [-201551, 21239, 107797, 107883, -201623, -201698, 225380, 408868, 408867, -201690]

In [27]:
df_exp = data_ventas_aux.loc[data_ventas_aux.padre_sku_id.isin(skus_exp)]

df_exp =  pd.merge(df_exp, df_margen2.loc[df_margen2.region == city_code].reset_index(drop=True),  how='left', left_on=['padre_sku_id'], right_on = ['id_tarjeta'])

df_exp['cash_margin'] = df_exp.gmv_usd * df_exp.GM

df_exp = df_exp.loc[:,['city','week','customer_id','name','padre_sku_id','category','subcat','orders','gmv_usd','discount_usd','cant','GM','cash_margin']]

In [28]:
#df_exp = pd.merge(df_exp, clear_buyers[['customer_id','business_type','is_kam']],  how='left', left_on=['customer_id'], right_on = ['customer_id'])
#df_exp = df_exp.loc[df_exp.is_kam == 0]

In [29]:
all_weeks = data_ventas_aux.groupby('week').customer_id.nunique()

In [30]:
all_weeks

week
09    5913
10    7155
11    7145
12    6704
13    4146
Name: customer_id, dtype: int64

In [31]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [32]:
def get_info(df_exp):
        df_check = (df_exp.groupby(by=['padre_sku_id','week','subcat'])
                .agg(customer_count=('customer_id', 'nunique'),
                orders_total=('orders', np.sum),
                gmv_usd=('gmv_usd', np.sum),
                discount_usd=('discount_usd', np.sum),
                quantity=('cant', np.sum),
                quantity_avg=('cant', np.mean),
                spend=('gmv_usd',np.mean),
                dct_avg=('discount_usd',np.mean),
                cash_margin=('cash_margin',np.sum)
                )
                .reset_index()
        )
        df_check['all_customers'] = df_check.week.apply(lambda x: all_weeks[f'{x}'])
        df_check['penetration'] = df_check.customer_count / df_check.all_customers
        df_check['ARPU'] = df_check.spend * df_check.penetration
        df_check['AOV'] = df_check.gmv_usd / df_check.orders_total
        df_check['GM'] = df_check.cash_margin / df_check.gmv_usd
        
        return df_check

In [33]:
def f0(df_exp,sku):
    #print('all /', df_exp.loc[df_exp.padre_sku_id == sku].name.unique()[0])
    a = get_info(df_exp.loc[(df_exp.padre_sku_id == sku) & 
                            (df_exp.customer_id.isin(clear_buyers[(clear_buyers.is_kam == 0) & (clear_buyers.business_type == "Restaurante")].customer_id.unique()))
                            ]
        )
    a['group'] = 'all'
    return a

In [34]:
def f1(df_exp,sku):
    #print('all no restaurant /', df_exp.loc[df_exp.padre_sku_id == sku].name.unique()[0])
    a = get_info(df_exp.loc[(df_exp.padre_sku_id == sku) &
                    (~df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_test'][0])
                        )
                    ) &
                    (~df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_all'][0])
                        )
                    )
                    ]
         )
    a['group'] = 'all no restaurant'
    return a

In [35]:
def f2(df_exp,sku):
    #print('all restaurant /', df_exp.loc[df_exp.padre_sku_id == sku].name.unique()[0])
    a=get_info(df_exp.loc[(df_exp.padre_sku_id == sku) &
                    (df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_all'][0])
                        )
                    ) &
                    (~df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_control'][0])
                        )
                    )
                    ]
         )
    a['group'] = 'all restaurant'
    return a

In [36]:
def f3(df_exp,sku):
    #print('test /', df_exp.loc[df_exp.padre_sku_id == sku].name.unique()[0])
    a = get_info(df_exp.loc[(df_exp.padre_sku_id == sku) &
                    (df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_test'][0])
                        )
                    )
                    ]
         )
    a['group'] = 'test'
    return a

In [37]:
def f4(df_exp,sku):
    #print('control /', df_exp.loc[df_exp.padre_sku_id == sku].name.unique()[0])
    a = get_info(df_exp.loc[(df_exp.padre_sku_id == sku) &
                    (df_exp.customer_id
                     .isin(
                        customers_offer((df_offers.loc[df_offers.card_id == sku,['customer_segment_id_all','customer_segment_id_test','customer_segment_id_control']].reset_index(drop=True).T)
                                        .loc['customer_segment_id_control'][0])
                        )
                    )
                    ]
         )
    a['group'] = 'control'
    return a

In [38]:
df = pd.DataFrame()#columns=['padre_sku_id','week','subcat','customer_count','orders_total','gmv_usd','discount_usd','quantity','quantity_avg','spend','dct_avg','all_customers','penetration','ARPU','AOV'])
for sku in skus_exp:
    df1 = pd.concat([f0(df_exp,sku), f1(df_exp,sku), f2(df_exp,sku), f3(df_exp,sku), f4(df_exp,sku)], ignore_index=True)
    df = pd.concat([df,df1], ignore_index=True)

In [39]:
df['net_cash_margin'] = df.cash_margin - df.discount_usd

In [40]:
df_final = df.loc[(df.group.isin(['control','test'])) & (df.week == '12')].reset_index(drop=True).copy()

In [41]:
df_final

,padre_sku_id,week,subcat,customer_count,orders_total,gmv_usd,discount_usd,quantity,quantity_avg,spend,dct_avg,cash_margin,all_customers,penetration,ARPU,AOV,GM,group,net_cash_margin
0,-201551,12,Aceites & Grasas,18,18,397.66555,3.97600,154.00,8.555556,22.092531,0.220889,38.932657,6704,0.002685,0.059318,22.092531,0.097903,test,34.956657
1,-201551,12,Aceites & Grasas,20,21,668.00295,13.36175,261.00,13.050000,33.400148,0.668087,65.399504,6704,0.002983,0.099642,31.809664,0.097903,control,52.037754
2,21239,12,Arroz,31,34,959.44308,9.59669,42.00,1.354839,30.949777,0.309571,81.467650,6704,0.004624,0.143115,28.218914,0.084911,test,71.870960
3,21239,12,Arroz,39,42,1256.41197,25.13459,55.00,1.410256,32.215692,0.644477,106.683692,6704,0.005817,0.187412,29.914571,0.084911,control,81.549102
4,107797,12,Enlatados,7,8,56.51295,0.56588,58.00,8.285714,8.073279,0.080840,3.539426,6704,0.001044,0.008430,7.064119,0.062630,test,2.973546
5,107797,12,Enlatados,5,6,55.31601,1.65952,57.00,11.400000,11.063202,0.331904,3.464461,6704,0.000746,0.008251,9.219335,0.062630,control,1.804941
6,107883,12,Enlatados,6,7,39.36792,0.39184,40.00,6.666667,6.561320,0.065307,2.515079,6704,0.000895,0.005872,5.623989,0.063887,test,2.123239
7,107883,12,Enlatados,6,6,56.34398,1.69260,58.00,9.666667,9.390663,0.282100,3.599620,6704,0.000895,0.008405,9.390663,0.063887,control,1.907020
8,-201623,12,Harinas & Mezclas,62,97,7520.00000,75.20000,188.00,3.032258,121.290323,1.212903,269.546787,6704,0.009248,1.121718,77.525773,0.035844,test,194.346787
9,-201623,12,Harinas & Mezclas,67,91,8000.00000,157.60000,200.00,2.985075,119.402985,2.352239,286.751901,6704,0.009994,1.193317,87.912088,0.035844,control,129.151901


# Calculate M1 - Inelasticity

In [42]:
def customers_offer_n(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.nunique()

In [43]:
def calculate_ROI(df):
    df.columns = df.iloc[17].unique()
    
    GMV_t = df['test'].loc['customer_count'] * df['test'].loc['spend'] * df['test'].loc['GM']# * margen_subcat
    dct_t = df['test'].loc['customer_count'] * df['test'].loc['dct_avg']

    GMV_c = df['control'].loc['customer_count'] * df['control'].loc['spend']  * df['control'].loc['GM']# * margen_subcat
    dct_c = df['control'].loc['customer_count'] * df['control'].loc['dct_avg']
    
    net_GMV_t = GMV_t - dct_t
    net_GMV_c = GMV_c - dct_c
    
    diff_net_GMV = net_GMV_t - net_GMV_c
    diff_net_dct = dct_c - dct_t
    
    ROI = diff_net_GMV / diff_net_dct
    return diff_net_GMV, diff_net_dct, ROI

In [44]:
for sku in df_final.padre_sku_id.unique():
    print(sku)
    diff_net_GMV, diff_net_dct, ROI = calculate_ROI(df_final.loc[df_final.padre_sku_id == sku].T)
    df_final.loc[df_final.padre_sku_id == sku,'diff_net_GMV'] = diff_net_GMV
    df_final.loc[df_final.padre_sku_id == sku,'diff_net_dct'] = diff_net_dct
    df_final.loc[df_final.padre_sku_id == sku,'ROI'] = ROI
    df_final.loc[df_final.padre_sku_id == sku,'region'] = city_code

-201551
21239
107797
107883
-201623
-201698
225380
408868
408867
-201690


In [45]:
df_final = pd.merge(df_final,data_ventas_aux[['padre_sku_id','name']].drop_duplicates(),  how='left', left_on=['padre_sku_id'], right_on = ['padre_sku_id'])
df_final = df_final[['region','group','padre_sku_id','name', 'week', 'subcat', 'all_customers', 'customer_count', 'orders_total',
    'gmv_usd', 'discount_usd', 'quantity', 'spend', 'dct_avg', 'quantity_avg', 'penetration', 'ARPU', 
    'AOV', 'GM', 'net_cash_margin', 'diff_net_GMV', 'diff_net_dct', 'ROI']]

In [46]:
df_final 

,region,group,padre_sku_id,name,week,subcat,all_customers,customer_count,orders_total,gmv_usd,...,dct_avg,quantity_avg,penetration,ARPU,AOV,GM,net_cash_margin,diff_net_GMV,diff_net_dct,ROI
0,CMX,test,-201551,Aceite Vegetal 1-2-3 x 1 lt,12,Aceites & Grasas,6704,18,18,397.66555,...,0.220889,8.555556,0.002685,0.059318,22.092531,0.097903,34.956657,-17.081097,9.38575,-1.819897
1,CMX,control,-201551,Aceite Vegetal 1-2-3 x 1 lt,12,Aceites & Grasas,6704,20,21,668.00295,...,0.668087,13.050000,0.002983,0.099642,31.809664,0.097903,52.037754,-17.081097,9.38575,-1.819897
2,CMX,test,21239,Arroz Súper Extra Sinaloa x 25 kg,12,Arroz,6704,31,34,959.44308,...,0.309571,1.354839,0.004624,0.143115,28.218914,0.084911,71.870960,-9.678142,15.53790,-0.622873
3,CMX,control,21239,Arroz Súper Extra Sinaloa x 25 kg,12,Arroz,6704,39,42,1256.41197,...,0.644477,1.410256,0.005817,0.187412,29.914571,0.084911,81.549102,-9.678142,15.53790,-0.622873
4,CMX,test,107797,Atún en Aceite Dolores x 133 gr,12,Enlatados,6704,7,8,56.51295,...,0.080840,8.285714,0.001044,0.008430,7.064119,0.062630,2.973546,1.168605,1.09364,1.068546
5,CMX,control,107797,Atún en Aceite Dolores x 133 gr,12,Enlatados,6704,5,6,55.31601,...,0.331904,11.400000,0.000746,0.008251,9.219335,0.062630,1.804941,1.168605,1.09364,1.068546
6,CMX,test,107883,Atún en Agua Dolores x 133 gr,12,Enlatados,6704,6,7,39.36792,...,0.065307,6.666667,0.000895,0.005872,5.623989,0.063887,2.123239,0.216219,1.30076,0.166225
7,CMX,control,107883,Atún en Agua Dolores x 133 gr,12,Enlatados,6704,6,6,56.34398,...,0.282100,9.666667,0.000895,0.008405,9.390663,0.063887,1.907020,0.216219,1.30076,0.166225
8,CMX,test,-201623,Harina de Trigo Espiga x 44 kg,12,Harinas & Mezclas,6704,62,97,7520.00000,...,1.212903,3.032258,0.009248,1.121718,77.525773,0.035844,194.346787,65.194886,82.40000,0.791200
9,CMX,control,-201623,Harina de Trigo Espiga x 44 kg,12,Harinas & Mezclas,6704,67,91,8000.00000,...,2.352239,2.985075,0.009994,1.193317,87.912088,0.035844,129.151901,65.194886,82.40000,0.791200


In [47]:
for offer_list in df_offers[['subcat','customer_segment_id_test','customer_segment_id_control']].values:
    df_final.loc[(df_final.subcat == offer_list[0]) & (df_final.group == 'test'),['all_exp_group']] = int(customers_offer_n(offer_list[1]))
    df_final.loc[(df_final.subcat == offer_list[0]) & (df_final.group == 'control'),['all_exp_group']] = int(customers_offer_n(offer_list[2]))

In [48]:
df_final

,region,group,padre_sku_id,name,week,subcat,all_customers,customer_count,orders_total,gmv_usd,...,quantity_avg,penetration,ARPU,AOV,GM,net_cash_margin,diff_net_GMV,diff_net_dct,ROI,all_exp_group
0,CMX,test,-201551,Aceite Vegetal 1-2-3 x 1 lt,12,Aceites & Grasas,6704,18,18,397.66555,...,8.555556,0.002685,0.059318,22.092531,0.097903,34.956657,-17.081097,9.38575,-1.819897,1.0
1,CMX,control,-201551,Aceite Vegetal 1-2-3 x 1 lt,12,Aceites & Grasas,6704,20,21,668.00295,...,13.050000,0.002983,0.099642,31.809664,0.097903,52.037754,-17.081097,9.38575,-1.819897,1.0
2,CMX,test,21239,Arroz Súper Extra Sinaloa x 25 kg,12,Arroz,6704,31,34,959.44308,...,1.354839,0.004624,0.143115,28.218914,0.084911,71.870960,-9.678142,15.53790,-0.622873,240.0
3,CMX,control,21239,Arroz Súper Extra Sinaloa x 25 kg,12,Arroz,6704,39,42,1256.41197,...,1.410256,0.005817,0.187412,29.914571,0.084911,81.549102,-9.678142,15.53790,-0.622873,240.0
4,CMX,test,107797,Atún en Aceite Dolores x 133 gr,12,Enlatados,6704,7,8,56.51295,...,8.285714,0.001044,0.008430,7.064119,0.062630,2.973546,1.168605,1.09364,1.068546,85.0
5,CMX,control,107797,Atún en Aceite Dolores x 133 gr,12,Enlatados,6704,5,6,55.31601,...,11.400000,0.000746,0.008251,9.219335,0.062630,1.804941,1.168605,1.09364,1.068546,85.0
6,CMX,test,107883,Atún en Agua Dolores x 133 gr,12,Enlatados,6704,6,7,39.36792,...,6.666667,0.000895,0.005872,5.623989,0.063887,2.123239,0.216219,1.30076,0.166225,85.0
7,CMX,control,107883,Atún en Agua Dolores x 133 gr,12,Enlatados,6704,6,6,56.34398,...,9.666667,0.000895,0.008405,9.390663,0.063887,1.907020,0.216219,1.30076,0.166225,85.0
8,CMX,test,-201623,Harina de Trigo Espiga x 44 kg,12,Harinas & Mezclas,6704,62,97,7520.00000,...,3.032258,0.009248,1.121718,77.525773,0.035844,194.346787,65.194886,82.40000,0.791200,263.0
9,CMX,control,-201623,Harina de Trigo Espiga x 44 kg,12,Harinas & Mezclas,6704,67,91,8000.00000,...,2.985075,0.009994,1.193317,87.912088,0.035844,129.151901,65.194886,82.40000,0.791200,263.0


In [49]:
df_final.to_csv(f'{city_code}_EXP_Hooks_Inelasticity.csv',index=False)
df.to_csv(f'{city_code}_EXP_Hooks_All_INFO_Inelasticity.csv',index=False)

# Calculate M2 - Elasticity

In [34]:
def customers_offer_n(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.nunique()

In [150]:
for offer_list in df_offers[['subcat','customer_segment_id_test','customer_segment_id_control']].values:
    df_final.loc[(df_final.subcat == offer_list[0]) & (df_final.group == 'test'),['all_exp_group']] = int(customers_offer_n(offer_list[1]))
    df_final.loc[(df_final.subcat == offer_list[0]) & (df_final.group == 'control'),['all_exp_group']] = int(customers_offer_n(offer_list[2]))

In [151]:
df_final['convertion_rate'] = df_final.customer_count/df_final.all_exp_group
df_final['AOV_net'] = df_final.gmv_usd * df_final.GM / df_final.customer_count

In [152]:
def calculate_ROI(df):
    df.columns = df.iloc[17].unique()
    #margen_subcat = df_margen3.loc[(df_margen3.region == df['test'].loc['region_code']) & (df_margen3.subcategoria == df['test'].loc['subcat'])].GM.unique()[0]
    
    GMV_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_exp_group'] * df['test'].loc['AOV_net']# * margen_subcat
    dct_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_exp_group'] * df['test'].loc['dct_avg']

    GMV_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_exp_group'] * df['control'].loc['AOV_net']# * margen_subcat
    dct_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_exp_group'] * df['control'].loc['dct_avg']
    
    net_GMV_t = GMV_t - dct_t
    net_GMV_c = GMV_c - dct_c
    
    diff_net_GMV = net_GMV_t - net_GMV_c
    diff_net_dct = dct_c - dct_t
    
    ROI = diff_net_GMV / diff_net_dct
    return diff_net_GMV, diff_net_dct, ROI

In [153]:
for card in df_final.padre_sku_id.unique():
    diff_net_GMV, diff_net_dct, ROI = calculate_ROI(df_final.loc[df_final.padre_sku_id == card].T)
    df_final.loc[df_final.padre_sku_id == card,'diff_net_GMV'] = diff_net_GMV
    df_final.loc[df_final.padre_sku_id == card,'diff_net_dct'] = diff_net_dct
    df_final.loc[df_final.padre_sku_id == card,'ROI'] = ROI
    df_final.loc[df_final.padre_sku_id == sku,'region'] = city_code

In [154]:
df_final.columns

Index(['padre_sku_id', 'week', 'subcat', 'customer_count', 'orders_total',
       'gmv_usd', 'discount_usd', 'quantity', 'quantity_avg', 'spend',
       'dct_avg', 'cash_margin', 'all_customers', 'penetration', 'ARPU', 'AOV',
       'GM', 'group', 'net_cash_margin', 'all_exp_group', 'convertion_rate',
       'AOV_net', 'diff_net_GMV', 'diff_net_dct', 'ROI', 'region'],
      dtype='object')

In [155]:
df_final = pd.merge(df_final,data_ventas_aux[['padre_sku_id','name']].drop_duplicates(),  how='left', left_on=['padre_sku_id'], right_on = ['padre_sku_id'])
df_final = df_final[['region','group','padre_sku_id','name', 'week', 'subcat', 'all_customers', 'customer_count', 'orders_total',
    'gmv_usd', 'discount_usd', 'quantity', 'spend', 'dct_avg', 'quantity_avg', 'penetration', 'ARPU', 
    'AOV', 'GM', 'net_cash_margin', 'all_exp_group', 'convertion_rate', 'AOV_net', 'diff_net_GMV', 'diff_net_dct', 'ROI']]

In [157]:
df_final.to_csv(f'{city_code}_EXP_Hooks_Elasticity.csv',index=False)
df.to_csv(f'{city_code}_EXP_Hooks_All_INFO_Elasticity.csv',index=False)